In [6]:
# RFM Model: Data Preparation

import pandas as pd

# Load the dataset (assuming it's in the same folder)
df = pd.read_csv("online_retail_listing.csv", sep=';', encoding='latin1')

# 1. Clean the data
df = df.dropna(subset=['Customer ID'])
df = df[df['Quantity'] > 0]
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], dayfirst=True, errors='coerce')

# 2. Create a 'TotalPrice' column
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['TotalPrice'] = df['Quantity'] * df['Price']

# 3. Set snapshot date (latest date in dataset + 1)
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

# 4. Aggregate RFM metrics
rfm = df.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'Invoice': 'nunique',
    'TotalPrice': 'sum'
})

# Rename columns
rfm.columns = ['Recency', 'Frequency', 'Monetary']

# 5. Remove negative monetary values (if any)
rfm = rfm[rfm['Monetary'] > 0]

# 6. Save RFM table
rfm.to_csv("rfm_scores.csv")

# Preview
print(rfm.head())


             Recency  Frequency  Monetary
Customer ID                              
12346.0          321         12       1.0
12348.0           71          5     361.0
12349.0           14          4     750.0
12350.0          305          1      40.0
12352.0           31         10     280.0
